**get model coordinates**

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

coor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/distan_lat42.58lon-8.8046p3R4Km.csv")
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

**get station and sample sd speed mean hour before**

In [2]:
station = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/station_data/coron.csv", usecols = ["time","dir_o"],parse_dates=["time"]).set_index("time")
print("raw data\n",station.head(24))

#show duplicated
df_duplicate = station[station.index.duplicated()]
print("duplicated")
print(df_duplicate)

#delete duplicated and resample
station = station[~station.index.duplicated(keep='first')]
station = station[station>0]
station = station.resample("H",label="right",closed="right").mean()

print("dir hour before")
print(station.head(4))
print("Variable statistics")
print(station.describe())

raw data
                      dir_o
time                      
2018-01-01 00:10:00  200.0
2018-01-01 00:20:00  198.0
2018-01-01 00:30:00  198.0
2018-01-01 00:40:00  197.0
2018-01-01 00:50:00  199.0
2018-01-01 01:00:00  199.0
2018-01-01 01:10:00  202.0
2018-01-01 01:20:00  205.0
2018-01-01 01:30:00  201.0
2018-01-01 01:40:00  199.0
2018-01-01 01:50:00  199.0
2018-01-01 02:00:00  202.0
2018-01-01 02:10:00  203.0
2018-01-01 02:20:00  199.0
2018-01-01 02:30:00  196.0
2018-01-01 02:40:00  196.0
2018-01-01 02:50:00  196.0
2018-01-01 03:00:00  198.0
2018-01-01 03:10:00  194.0
2018-01-01 03:20:00  197.0
2018-01-01 03:30:00  209.0
2018-01-01 03:40:00  218.0
2018-01-01 03:50:00  217.0
2018-01-01 04:00:00  207.0
duplicated
                     dir_o
time                      
2018-03-25 03:00:00   54.0
2018-03-25 03:10:00   28.0
2018-03-25 03:20:00   38.0
2018-03-25 03:30:00   45.0
2018-03-25 03:40:00   44.0
2018-03-25 03:50:00   44.0
2019-03-31 03:00:00   49.0
2019-03-31 03:10:00   53.0
2019-03

**get meteorologic model d0**

In [3]:
#ria forecast d0
met_model = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/lat42.58lon-8.8046p3R4KmD0.csv",parse_dates=["time"]).drop(columns=["Unnamed: 0"]).set_index("time")
met_model.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42021 entries, 2018-01-01 01:00:00 to 2023-01-01 23:00:00
Data columns (total 75 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dir0         42021 non-null  float64
 1   snow_prec0   42021 non-null  float64
 2   snowlevel0   42021 non-null  float64
 3   mod0         42021 non-null  float64
 4   wind_gust0   42021 non-null  float64
 5   mslp0        42021 non-null  float64
 6   temp0        42021 non-null  float64
 7   rh0          42021 non-null  float64
 8   visibility0  42021 non-null  float64
 9   lhflx0       42021 non-null  float64
 10  lwflx0       42021 non-null  float64
 11  conv_prec0   42021 non-null  float64
 12  prec0        42021 non-null  float64
 13  swflx0       42021 non-null  float64
 14  shflx0       42021 non-null  float64
 15  cape0        42021 non-null  float64
 16  cin0         42021 non-null  float64
 17  cfh0         42021 non-null  float64
 18  cfl0       

**Concatenate and dropnan**

In [4]:
df_all = pd.concat([station,met_model],axis=1).dropna()
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41373 entries, 2018-01-01 01:00:00 to 2022-12-30 23:00:00
Data columns (total 76 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dir_o        41373 non-null  float64
 1   dir0         41373 non-null  float64
 2   snow_prec0   41373 non-null  float64
 3   snowlevel0   41373 non-null  float64
 4   mod0         41373 non-null  float64
 5   wind_gust0   41373 non-null  float64
 6   mslp0        41373 non-null  float64
 7   temp0        41373 non-null  float64
 8   rh0          41373 non-null  float64
 9   visibility0  41373 non-null  float64
 10  lhflx0       41373 non-null  float64
 11  lwflx0       41373 non-null  float64
 12  conv_prec0   41373 non-null  float64
 13  prec0        41373 non-null  float64
 14  swflx0       41373 non-null  float64
 15  shflx0       41373 non-null  float64
 16  cape0        41373 non-null  float64
 17  cin0         41373 non-null  float64
 18  cfh0       

**LGBMRegressor**

In [5]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.dir_o
X = df_all[df_all.columns[1:]]

MAE = []
for t in range (0,10):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE":MAE,});
df_r.describe()


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059462 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18213
[LightGBM] [Info] Number of data points in the train set: 37235, number of used features: 72
[LightGBM] [Info] Start training from score 141.939435
machine learning number: 0
            y_test       y_pred         diff
count  4138.000000  4138.000000  4138.000000
mean    142.164109   142.771831     0.607722
std      89.682883    75.292742    48.116394
min       4.333333     0.304347  -273.553512
25%      49.166667    62.864272   -13.981310
50%     159.500000   153.135355     1.358342
75%     215.625000   209.363447    17.797196
max     355.666667   298.697386   229.103818
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035011 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18224
[LightGBM] [Info] Number o

Mean absolute value all machine learning models


,MAE
count,10.00000
mean,30.13100
std,0.41549
min,29.58000
25%,29.92250
50%,29.96500
75%,30.40250
max,30.78000


**Shapiro test normality MAE samples tested**

In [6]:
from scipy.stats import shapiro

sta, p = shapiro(MAE)
print("\np value:",p)
print("significance level alpha=0.01")

alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable MAE in the samples")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable MAE no normal distribution in samples")


p value: 0.32368892431259155
significance level alpha=0.01
Accept null hypothesis: normal distribution variable MAE in the samples


**Add time variables**

In [7]:

df_all["hour"] = df_all.index.hour
df_all["month"] = df_all.index.month
df_all["dayofyear"] = df_all.index.dayofyear
df_all["weekofyear"] = df_all.index.isocalendar().week.astype(int)

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.dir_o
X = df_all[df_all.columns[1:]]


MAE_t = []
for t in range (0,10):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE_t.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE_t":MAE_t,});
df_r.describe()



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062037 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18553
[LightGBM] [Info] Number of data points in the train set: 37235, number of used features: 76
[LightGBM] [Info] Start training from score 141.939435
machine learning number: 0
            y_test       y_pred         diff
count  4138.000000  4138.000000  4138.000000
mean    142.164109   142.821606     0.657497
std      89.682883    75.927419    47.127463
min       4.333333    20.337431  -245.268200
25%      49.166667    62.032841   -13.508704
50%     159.500000   152.724819     1.357285
75%     215.625000   210.952134    17.226047
max     355.666667   305.077410   233.735371
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037770 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18565
[LightGBM] [Info] Number o

Mean absolute value all machine learning models


,MAE_t
count,10.000000
mean,29.298000
std,0.426844
min,28.550000
25%,29.117500
50%,29.215000
75%,29.352500
max,30.130000


**T test two related samples. This is a test for the null hypothesis that two related samples have identical mean absolute error. Alternative hypothesis the  mean absolute error from the sample with time variables is less than the mean absolute error of from the sample with time variables. Significance level (alpha) = 0.01**

In [8]:
from scipy import stats

alpha = 0.01
sta, p_value = stats.ttest_rel(MAE, MAE_t, alternative="greater")

print("\nMAE without time variables:", np.round(MAE,3))
print("\nMAE with time variables:", np.round(MAE_t,3))
print("\nalpha: ",alpha)
print("\np_value point: ",p_value)

if p_value < alpha:
    print("\nwe are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables")
    print("\nwe are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs")
else:
    print("\nwe are accepting null hypothesis:  H0: MAE machine learning without time variables = MAE machine learning with time variables")


MAE without time variables: [29.94 30.78 29.67 30.38 30.71 29.58 29.99 29.92 30.41 29.93]

MAE with time variables: [29.11 30.13 29.11 29.24 29.82 28.55 29.19 29.14 29.36 29.33]

alpha:  0.01

p_value point:  1.5300593204599297e-07

we are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables

we are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs


**Save algorithm**

In [9]:
import pickle
from sklearn.pipeline import Pipeline


estimators = [('ml_model', ml_model)]
pipe = Pipeline(estimators)
print(pipe)
al_file={"x_var":X.columns,"coor":coor,"pipe":pipe}
pickle.dump(al_file, open("/content/drive/MyDrive/Colab Notebooks/Coron/algorithms/rdir_coron_d0.al", 'wb'))

Pipeline(steps=[('ml_model', LGBMRegressor(n_estimators=200))])
